# Mixed-effects Models in `R`
As the final part of this lesson, we turn to the process of fitting LME models using `R`, as well as options around inference. These examples will all be fairly basic, but we will see more variety in the workshop this week. In addition, we will see that there are *two* main options in `R` for fitting LME models. The first is the `lme()` function from the `nlme` package and the second is the `lmer()` function from the `lme4` package. We will discuss both below, as well as their advantages and disadvantages.

## Fitting Mixed-effects Models with `nlme`
We will start with the `nlme` package, both because it is the *older* of the two and because it is already familiar from the `gls()` function. To fit a LME model, we use the `lme()` function, which has a syntax very similar to the `gls()` function, but with the addition of a `random=` option for specifying the random effects. To understand how to use this, we will return to the long-formatted `selfesteem` data from `datarium`.

In [2]:
library('datarium')
library('reshape2')

data('selfesteem')

# repeats and number of subjects
t <- 3
n <- dim(selfesteem)[1]

# reshape wide -> long
selfesteem.long <- melt(selfesteem,            # wide data frame
                        id.vars='id',          # what stays fixed?
                        variable.name="time",  # name for the new predictor
                        value.name="score")    # name for the new outcome

selfesteem.long           <- selfesteem.long[order(selfesteem.long$id),] # order by ID
rownames(selfesteem.long) <- seq(1,n*t)                                  # fix row names
selfesteem.long$id        <- as.factor(selfesteem.long$id)               # convert ID to factor

In [39]:
library('datarium')
library('reshape2')

data('anxiety')
anxiety.grp3 <- anxiety[anxiety$group == 'grp3',]   # high exercise group
anxiety.grp3 <- subset(anxiety.grp3, select=-group) # remove group column

# repeats and number of subjects
t <- 3
n <- dim(anxiety.grp3)[1]

# reshape wide -> long
anxiety.long <- melt(anxiety.grp3,             # wide data frame
                     id.vars='id',             # what stays fixed?
                     variable.name="time",     # name for the new predictor
                     value.name="score")       # name for the new outcome

anxiety.long           <- anxiety.long[order(anxiety.long$id),] # order by ID
rownames(anxiety.long) <- seq(1,n*t)                            # fix row names
anxiety.long$id        <- as.factor(anxiety.long$id)            # convert ID to factor


In [40]:
head(anxiety.long)

  id time score
1 31   t1  14.6
2 31   t2  13.0
3 31   t3  11.7
4 32   t1  15.0
5 32   t2  13.0
6 32   t3  11.9

In [41]:
lmer.mod <- lmer(score ~ time + (1|id), data=anxiety.long)
print(lmer.mod)

Linear mixed model fit by REML ['lmerMod']
Formula: score ~ time + (1 | id)
   Data: anxiety.long
REML criterion at convergence: 84.4371
Random effects:
 Groups   Name        Std.Dev.
 id       (Intercept) 1.3445  
 Residual             0.3034  
Number of obs: 45, groups:  id, 15
Fixed Effects:
(Intercept)       timet2       timet3  
     17.013       -2.000       -3.453  


In [5]:
head(selfesteem.long)

  id time    score
1  1   t1 4.005027
2  1   t2 5.182286
3  1   t3 7.107831
4  2   t1 2.558124
5  2   t2 6.912915
6  2   t3 6.308434

Our multilevel model for these data was specified earlier as

$$
\begin{alignat*}{2}
    y_{ij}    &= \mu_{i} + \alpha_{j} + \eta_{ij} &\quad\text{Level 1} \\
    \mu_{i}   &= \mu + \xi_{i}                    &\quad\text{Level 2} \\
\end{alignat*}
$$

which we can collapse to a single level by replacing $\mu_{i}$ with $\mu + \xi_{i}$. This gives the LME model

$$
y_{ij} = \mu + \alpha_{j} + \xi_{i} + \eta_{ij},
$$

where we have *two* fixed-effects of $\mu$ and $\alpha_{j}$, and *two* random-effects of $\xi_{i}$ and $\eta_{ij}$. The *fixed-effects* are used to construct the *marginal* mean structure across all subjects, whereas the *random-effects* are used to construct the *marginal* variance-covariance structure across all subjects. 

To fit this model using `lme()` we use

In [7]:
library(nlme)
lme.mod <- lme(score ~ time, random= ~ 1|id, data=selfesteem.long)

So, the key here is understanding the `random=` syntax. This has a form very similar to what we have seen before when using `correlation=` and `weights=` in the `gls()` function. However, we need to think about this a little differently because we are not defining how we want a particular covariance structure to apply to the data. Instead, we are giving the *structure* of the random effects. 

We can think of this as simply a direct implementation of the LME equation above. We have

$$
y_{ij} = \overbrace{\mu + \alpha_{j}}^{\text{fixed}} + \overbrace{\xi_{i} + \eta_{ij}}^{\text{random}},
$$

so the *fixed-effects* part just appears in the model equation as `score ~ time` or, alternatively, `score ~ 1 + time`. The *random-effects* part consists of $\xi_{i} + \eta_{ij}$. Now, we do not need to specify $\eta_{ij}$ because that is the residual error term that is *always* included in a linear model. We know this already from `lm()` as we never have to ask for residuals, they are always there. So, the only thing that differentiates the model above from something we could fit with `lm()` is the term $\xi_{i}$. This is *precisely* what we specify using the `random=` argument. 

So what is $\xi_{i}$? Well, if we look at its index, it is a value that changes with *each subject*. It is therefore *constant* within a subject, but different between subjects. So how do we specify a value that is only constant *within* each subject? Well, we represent it as a constant using a `1` but make it *conditional* on the subject by using `|id`. So `1|id` can be read as *a constant per-subject*. This is why this type of model is also known as a *random intercept* model.

Alternatively, we can go back to the multilevel conceptualisation and think of this as describing the model that we fit to each individual subject. We will use some pseudo-code below to get the idea across, just be aware that most of this is not valid `lme()` syntax (but the ideas are hopefully clear). As described earlier, when we have *no replications* within each level of the repeated measures, the best we can do with the data from *one* subject is fit a constant. So our model for one subject would be something like

`score[id == 1] ~ 1`

If we wanted this to work for *all* subjects, we would need to indicate that we want a *different* constant for each subject, which would give something like

`score ~ 1|id`

which we can read as "fit a constant *separately* for each level of `id`". 

Once we have multiple subjects, we have the option of estimating values by pooling data *across* subjects. This only works if we assume we are estimating something that is common across the subjects, so we imagine all subjects are drawn from the same population with the same mean. This is where the *fixed-effects* come in, as a parameterisation of the population mean. So, with multiple subjects, we can add a fixed-effect of `time` in the form $\mu + \alpha_{j}$. Using our current pseudo-code, this would give

`score ~ 1|id + 1 + time`.

However, this is *not* how `lme()` works. Instead, in order to clearly differentiate the random-effects from the fixed-effects, `lme()` requires that we move anything random out of the model equation and put it in a separate argument. So this gives the final *real* specification of

`lme(score ~ 1 + time, random= ~ 1|id)`

In [10]:
print(lme.mod)

Linear mixed-effects model fit by REML
  Data: selfesteem.long 
  Log-restricted-likelihood: -38.49673
  Fixed: score ~ time 
(Intercept)      timet2      timet3 
   3.140122    1.793820    4.496220 

Random effects:
 Formula: ~1 | id
         (Intercept)  Residual
StdDev: 2.772358e-05 0.8859851

Number of Observations: 30
Number of Groups: 10 


## Fitting Mixed-effects Models with `lme4`


In [9]:
library(lme4)
lmer.mod <- lmer(score ~ time + (1|id), data=selfesteem.long)

print(VarCorr(lmer.mod))

boundary (singular) fit: see help('isSingular')
 Groups   Name        Std.Dev.
 id       (Intercept) 0.00000 
 Residual             0.88599 




... The nice thing about this is that the random effects *do* appear in the model equation. They are simply differentiated using round brackets. So we can use

`score ~ (1|id) + 1 + time`

though it is more usual to specify the fixed-effects *first* and the random-effects *second*. This also helps will conceptualising the random-effects as *addition error terms*. This gives the final *real* specification of

`lmer(score ~ 1 + time + (1|id))`

In [16]:
gls.mod <- gls(score ~ time, correlation=corSymm(form = ~1|id), data=selfesteem.long)
summary(gls.mod)

Generalized least squares fit by REML
  Model: score ~ time 
  Data: selfesteem.long 
       AIC      BIC    logLik
  87.34893 96.41979 -36.67447

Correlation Structure: General
 Formula: ~1 | id 
 Parameter estimate(s):
 Correlation: 
  1      2     
2 -0.310       
3  0.279 -0.440

Coefficients:
               Value Std.Error   t-value p-value
(Intercept) 3.140122 0.2773446 11.322095   0e+00
timet2      1.793820 0.4488710  3.996293   4e-04
timet3      4.496220 0.3331042 13.497939   0e+00

 Correlation: 
       (Intr) timet2
timet2 -0.809       
timet3 -0.601  0.304

Standardized residuals:
        Min          Q1         Med          Q3         Max 
-1.51407798 -0.63430904 -0.03250738  0.45770104  2.44238240 

Residual standard error: 0.8770406 
Degrees of freedom: 30 total; 27 residual

## Inference in Mixed-effects Models

### The Implied Marginal Model
... From this perspective, a mixed-effects model is simply GLS, but with a more sophisticated way of constructing the variance-covariance matrix using the random effects. This leads to some important consequences:

1. The inferential issues around GLS are *identical* for mixed-effects
2. The covariance structure is *more restricted* under mixed-effects, because it is constrained by the random effects themselves 

... The advantages of GLS are still there. For instance, the correct error term for each test can be automatically generated from the covariance structure. So while the model itself may be no better than the repeated measures ANOVA, it is infinitely *more practical* because we do not need to mess around manually assigning error terms to different tests (and worrying about getting this wrong).

### Kenward-Rogers Degrees of Freedom
... The advantage is that mixed-effects models capture more *structure* than GLS models. This means that mixed-effects models embed *where* the correlation came from. Having that structure in terms of the random effects allows for slightly more sophisticated approaches to the problem of *effective degrees of freedom*. Because the covariance structure can be decomposed into the random effects, the data structure itself is part of the model of the covariance matrix. This means there is information available that can be used to construct more sophisticated faux degrees of freedom. This information is not there using GLS, because there is only a single big covariance structure that cannot be decomposed into anything related to the model. 

Now, of course, this is only an advantage if you agree with the notion of effective degrees of freedom. If you do not, then this new information is still not useful.

## Inference Using `nlme`

## Inference Using `lme4`

`lmerTest` package